# MNIST Data Exploration

#### About the data
* handwritten digits (0-9)
* each image 28 pixels x 28 pixels = 784 pixels in total
* pixel value is between 0 and 255, higher numbers meaning darker
* train data has 785 columns. First column: label, 784 columns: pixel values

## Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import Data

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')